In [ ]:
import sys

sys.path.append("../")

In [ ]:
import uuid

from sqlmodel import Session, create_engine

from app.database import create_db_and_tables
from app.models import Discussion, Message
from app.settings import settings
from app.types import Category

In [ ]:
engine = create_engine(
    f"postgresql+psycopg://{settings.database_user}:{settings.database_password.get_secret_value()}@{settings.database_host}:{settings.database_port}/{settings.database_name}",
)

create_db_and_tables(engine)

In [ ]:
discussions = [
    {
        "template": "BINARY_PROPOSAL",
        "name": "",
        "description": "",
        "tags": ["tag1", "tag2"],
        "messages": ["Hello World"],
    },
    {
        "template": "FEEDBACK_RETROSPECTIVE",
        "name": "sustainability",
        "description": "",
        "tags": ["tag1", "tag2"],
        "messages": ["Hello World"],
    },
]

In [ ]:
with Session(engine) as session:
    for discussion in discussions:
        db_discussion = Discussion(
            template=Category(discussion["template"]),
            name=discussion["name"],
            description=discussion["description"],
            tags=set(discussion["tags"]),
            owner_id=uuid.uuid4(),
        )

        session.add(db_discussion)
        session.commit()
        session.refresh(db_discussion)

        for message in discussion["messages"]:
            db_message = Message(message=message, discussion_id=db_discussion.id, owner_id=uuid.uuid4())
            session.add(db_message)
            session.commit()